In [14]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_rows', 100)
import missingno as msno
import re
from sklearn.preprocessing import LabelEncoder
import pickle
%matplotlib inline

In [21]:
train_df = pd.read_csv("../input/train.csv")
test_df = pd.read_csv("../input/test.csv")

C:\Program Files\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (45) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [22]:
train_df.head(6).transpose()

,0,1,2,3,4,5
member_id,13599771,1491956,64687987,8446850,58471388,5337525
loan_amnt,8875,12500,33600,17000,14000,24000
funded_amnt,8875,12500,33600,17000,14000,24000
funded_amnt_inv,8875,12475,33600,17000,14000,24000
term,36 months,36 months,60 months,60 months,60 months,60 months
int_rate,18.25,8.9,18.55,9.71,16.99,17.27
installment,321.97,396.92,863.31,358.78,347.87,599.96
grade,D,A,E,B,D,C
sub_grade,D3,A5,E2,B1,D3,C5
emp_title,Inventory/Part time Sup,Bel Brands USA,Assistant Principal,Cook Childrens Medical Center,Cook,walmart distribution cntr


In [ ]:
print (train_df.info())

In [ ]:
#目前使用的特征
feature = ['loan_amnt',#借贷总额
           'funded_amnt',
           'funded_amnt_inv',
           'term', 
           'int_rate',
           'installment',
           'grade',
           'sub_grade',
           'emp_length',#从业时长
           'home_ownership',
           'annual_inc',#年收入,有4个缺失值
           'verification_status',#object
           'loan_status',#object
           'pymnt_plan',#object,是否已经为贷款实施了付款计划
           'dti',#月付与月收比率
           'mths_since_last_record',#有大量缺失值
           'pub_rec',#有缺失值
           'revol_bal',
           'revol_util',
           'total_acc',#信用额度
           'initial_list_status',#object
           'out_prncp',
           'out_prncp_inv',
           'total_pymnt',
           'total_pymnt_inv',
           'total_rec_prncp',
           'total_rec_int',
           'total_rec_late_fee',
           'recoveries']

In [23]:
#Combine into data:
train_df['source']= 'train'
test_df['source'] = 'test'
dataset=pd.concat([train_df,test_df],ignore_index=True)
dataset.shape

(887379, 65)

In [24]:
# Check missing:
print(dataset.isnull().sum())

acc_now_delinq                 177476
addr_state                          0
all_util                       866007
annual_inc                          4
annual_inc_joint               886868
application_type                    0
collection_recovery_fee             0
collections_12_mths_ex_med        145
desc                           761350
dti                                 0
dti_joint                      886870
earliest_cr_line                   29
emp_length                          0
emp_title                       51457
funded_amnt                         0
funded_amnt_inv                     0
grade                               0
home_ownership                      0
il_util                        868762
initial_list_status                 0
inq_fi                         866007
inq_last_12m                   866007
installment                         0
int_rate                            0
issue_d                             0
loan_amnt                           0
loan_status 

In [25]:
# XGboost is good at dealing with numbers but definitely not good when dealing with string. 
# So the solution is neither we drop them or transform them.
drop_elements = ['annual_inc_joint',# high missing rate
                 'dti_joint',# high missing rate
                 'verification_status_joint',# high missing rate
                 'issue_d',
                 'emp_title',
                 'desc',
                 'title',
                 'desc', 
                 'zip_code',
                 'addr_state',
                 'earliest_cr_line', 
                 'application_type']
dataset.drop(drop_elements,axis=1,inplace=True)

# Mapping emp_length
emp_length_mapping = {'< 1 year': 1, '1 year': 2, '2 years': 3, '3 years': 4, '4 years': 5, '5 years': 6, '6 years': 7, '7 years': 8, '8 years': 9, '9 years': 10, '10 years': 11, '10+ years': 12}
dataset['emp_length'] = dataset['emp_length'].map(emp_length_mapping)
dataset['emp_length'] = dataset['emp_length'].fillna(0)

# Mapping term: 36 months->36
dataset['term']=dataset['term'].apply(lambda row:int(re.findall(r"\d+\.?\d*",row)[0]))

# Mapping Category feature
category_feature=['grade','sub_grade','home_ownership','verification_status','pymnt_plan','loan_status','purpose','initial_list_status']
dataset[category_feature]=dataset[category_feature].apply(LabelEncoder().fit_transform)
    
#Handle datetime，将Feb-1985转换成1985-02-01
#     dataset['earliest_cr_line'] =  pd.to_datetime(dataset['earliest_cr_line'], format='%b-%Y')
#     dataset['issue_d'] = pd.to_datetime(dataset['issue_d'], format='%b-%Y')

In [ ]:
print (train_df.info())

In [ ]:
train_df.head().transpose()

In [26]:
train = dataset.loc[dataset['source']=='train']
test = dataset.loc[dataset['source']=='test']

In [27]:
train.drop('source',axis=1,inplace=True)
test.drop(['source','acc_now_delinq'],axis=1,inplace=True)

C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [29]:
train.to_csv('../dump/train_modified.csv',index=False)
test.to_csv('../dump/test_modified.csv',index=False)